In [1]:
from pypdf import PdfReader

In [2]:
pdf_reader = PdfReader('./Civics.pdf')

In [3]:
data = []
for i, page in enumerate(pdf_reader.pages):
    data.append({'page':i, 'content':page.extract_text()})

In [4]:
data = data[6:len(pdf_reader.pages)]

In [5]:
import pandas as pd

In [6]:
df = pd.DataFrame(data)

In [7]:
import nltk
import string
pd.options.mode.chained_assignment = None

In [8]:
df['text_lower'] = df['content'].str.lower()

In [9]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    return text.translate(str.maketrans('','', PUNCT_TO_REMOVE))

df['text_wo_punct'] = df['text_lower'].apply(lambda text: remove_punctuation(text))


In [10]:
import chromadb
import sentence_transformers

In [11]:
from sentence_transformers import SentenceTransformer, util

In [12]:
model = SentenceTransformer('all-MiniLM-L6-v2')

2023-09-19 12:29:50.960789: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
documents = []
metadatas = []
ids = []

for i in range(len(df)):
    documents.append(df['text_wo_punct'][i][6:])
    metadatas.append({'unit':df['text_wo_punct'][i][:6], 'page':i + 1})
    ids.append(str(i + 3))


In [14]:
client = chromadb.Client()

In [17]:
vector_coll = client.create_collection('civics-latest-1.0.0')

vector_coll.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [20]:
results = vector_coll.query(
    query_texts=['equality'],
    n_results=5
)
for i in range(len(results)):
    print('Ids:' , results['ids'][0][i])
    print('Metadatas:', results['metadatas'][0][i])
    print('Description:', results['documents'][0][i])
    print('\n')

Ids: 48
Metadatas: {'page': 46, 'unit': 'unit 3'}
Description:  — equality lesson 1  
 48 square6 civics and ethical education — grade 10 student textbookby the end of this lesson you should be able to
 circle6 explain the new dimensions of equality after 
the transition to democracy in ethiopia 
do you have some idea about the history of 
equality in ethiopia
you learnt the meaning of equality in grade 9 
equality does not mean the provision of identical treatment and benefits when we say all citizens are equal we are not saying all of them should receive identical treatment and rewards for example hundreds of athletes may participate in a marathon competition where the rules and the course are the same for every competitor but depending on their ability and performance they may get different awards absolute equality and similarity do not exist between any two people this is because we differ in physical strength intellectual capacity interest and devotion 
 equality means having simi